In [1]:
%load_ext sql
%sql sqlite:///store.db

In [2]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

## Задание 1
Вывести страну, где популярнее всего группа Iron Maiden (т.е. треков куплено больше, чем в других странах). Также вывести кол-во купленных треков.

In [13]:
%%sql
SELECT c.Country as Country, SUM(ii.Quantity) as tracks_count
FROM tracks t
JOIN invoice_items ii ON t.TrackId = ii.TrackId
JOIN invoices i ON ii.InvoiceId = i.InvoiceId
JOIN customers c ON i.CustomerId = c.CustomerId
JOIN albums a ON t.AlbumId = a.AlbumId
JOIN artists ar ON a.ArtistId = ar.ArtistId
WHERE ar.Name like 'Iron Maiden'
GROUP BY Country
ORDER BY COUNT(t.TrackId) DESC
LIMIT 1

 * sqlite:///store.db
Done.


Country,tracks_count
USA,61


## Задание 2
Вывести "средний по длине" трек, среди приобретённых компаниями (только клиентами из конкретных компаний) после 22 апреля 2023 года.
"Средний по длине" = максимально близкий к среднему значению множества треков по длине (не медиана!). Если таких треков несколько, вывести первый по алфавиту.

In [19]:
%%sql
SELECT t.Name as track_name, t.Milliseconds as track_length
FROM tracks t
JOIN invoice_items ii ON t.TrackId = ii.TrackId
JOIN invoices i ON ii.InvoiceId = i.InvoiceId
JOIN customers c ON i.CustomerId = c.CustomerId
WHERE c.Company IS NOT NULL AND i.InvoiceDate > '2023-04-22'
ORDER BY ABS(t.Milliseconds - (SELECT AVG(Milliseconds) FROM tracks)) and t.Name
LIMIT 1

 * sqlite:///store.db
Done.


TrackId,Name,Milliseconds
1601,No Quarter,420493


## Задание 3

Вывести треки, встречающиеся и в нескольких плейлистах, и в нескольких заказах одновременно.

In [20]:
%%sql
SELECT t.Name as track_name
FROM tracks t
JOIN playlist_track pt ON t.TrackId = pt.TrackId
JOIN invoice_items ii ON t.TrackId = ii.TrackId
GROUP BY t.TrackId
HAVING COUNT(DISTINCT pt.PlaylistId) > 1 AND COUNT(DISTINCT ii.InvoiceLineId) > 1

 * sqlite:///store.db
Done.


track_name
Sunshine Of Your Love
The Woman King
The Fix
Walkabout
Hot Girl
Gay Witch Hunt
Phyllis's Wedding
How to Stop an Exploding Man
Pilot


## Задание 4
Вывести альбом, который является максимальным по продолжительности среди альбомов со стоимостью, большей средней цены за альбом *(цена альбома = сумма цен треков, продолжительность считать аналогично)*.

In [28]:
%%sql
SELECT a.Title AS album_title
FROM albums a
JOIN tracks t ON a.AlbumId = t.AlbumId
GROUP BY a.AlbumId
HAVING
    SUM(t.UnitPrice) > (
        SELECT AVG(album_price)
        FROM (
            SELECT SUM(t.UnitPrice) AS album_price
            FROM albums a
            JOIN tracks t ON a.AlbumId = t.AlbumId
            GROUP BY a.AlbumId
        )
    )
ORDER BY SUM(t.Milliseconds) DESC
LIMIT 1;


 * sqlite:///store.db
Done.


album_title
"Lost, Season 3"


## Задание 5
Для каждого исполнителя, у которого есть песни нескольких жанров, найти жанр, по которому он заработал наибольшее количество денег, а также вывести процентное соотношение суммы, заработанной исполнителем по этому жанру к общей сумме заработанных денег. Формат вывода: имя исполнителя, жанр, соотношение.

In [47]:
%%sql
SELECT a.Name as Name, g.Name as Genre,
FROM artists a
JOIN albums al ON a.ArtistId = al.ArtistId
JOIN tracks t ON al.AlbumId = t.AlbumId
JOIN genres g ON t.GenreId = g.GenreId
JOIN invoice_items ii ON t.TrackId = ii.TrackId
JOIN invoices i ON ii.InvoiceId = i.InvoiceId
GROUP BY a.Name
HAVING COUNT(DISTINCT t.GenreId) > 1

 * sqlite:///store.db
Done.


Name,Genre,Percentage
Amy Winehouse,R&B/Soul,100.0
Antônio Carlos Jobim,Jazz,100.0
Audioslave,Rock,100.0
Battlestar Galactica,TV Shows,100.0
Eric Clapton,Blues,100.0
Faith No More,Alternative & Punk,100.0
Foo Fighters,Rock,100.0
Gilberto Gil,Soundtrack,100.0
Guns N' Roses,Rock,100.0
Heroes,Drama,100.0
